# Preparazione data set per i test
Loading dei dati necessari ai test e suddivisione in 3 Dataframe: 
* dfLow = zona a bassa coverage;
* dfNormal = zona a media coverage
* dfHigh = zona ad alta coverage

In [1]:
#scrittura df per test
import pandas as pd
from shapely.geometry import Point, Polygon,box
import yaml
import geopandas as gpd
#import cartopy.geodesic as GD
import numpy as np
from datetime import datetime
import os


with open("../conf.yaml") as f:
    conf = yaml.load(f, Loader = yaml.FullLoader)

pathBox = conf["pathBox"]

# Check and read the start end period period for computing the coverage. 
if "coverage_start_period" and "coverage_end_period" in conf:
    coverage_start_period = conf["coverage_start_period"]
    coverage_end_period = conf["coverage_end_period"]
else:
    # coverage period overlaps with the start and end time of the data set
    coverage_start_period = conf["start_time"]
    coverage_end_period = conf["end_time"]

start_time_obj = datetime.strptime(coverage_start_period, '%Y-%m-%d %H:%M:%S')
end_time_obj = datetime.strptime(coverage_end_period, '%Y-%m-%d %H:%M:%S')
a = str(start_time_obj.year)+"_"+str(start_time_obj.month)+"_"+str(start_time_obj.day)
b = str(end_time_obj.year)+"_"+str(end_time_obj.month)+"_"+str(end_time_obj.day)
# build suffix for the output file namnes
suffix = a+"-"+b
out_path = conf["out_path"]

coverage_path = "../"+out_path+ "scenario/mix/"+str(conf["scale_value"])+"/coverages_"+suffix+".csv"
coverage_path





'../../output/coverage/scenario/mix/10.0/coverages_2008_7_1-2009_12_31.csv'

In [2]:
# build the folders 
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
# build output paths
pathDFLow = conf["pathDFLow"]+suffix+"/"
pathDFNormal = conf["pathDFNormal"]+suffix+"/"
pathDFHigh = conf["pathDFHigh"]+suffix+"/"
pathDFBorder = conf["pathDFBorder"]+suffix+"/"

ensure_dir(pathDFLow)
ensure_dir(pathDFNormal)
ensure_dir(pathDFHigh)
ensure_dir(pathDFBorder)



In [3]:
#read the coverage file
GeneralDf = pd.read_csv(coverage_path, delimiter=',')
#rinomino le colonne
#GeneralDf.rename(columns={'Unnamed: 0': 'index', 'Unnamed: 0.1' : 'index2'}, inplace=True)
#elimino le colonne che non servono
#GeneralDf.drop(GeneralDf.columns[[0,1,2,3,6]], axis='columns', inplace=True)
print(GeneralDf.head())

#trasformo il dataframe in un gedoDataFrame
gdf = gpd.GeoDataFrame(GeneralDf,crs="EPSG:4326", geometry=gpd.points_from_xy(GeneralDf.lon, GeneralDf.lat))
gdf = gdf.drop(['lon', 'lat'], axis=1)
print(gdf.head())

#lettura dei box
Box = gpd.GeoDataFrame(gpd.read_file(pathBox),crs="EPSG:4326")
Box=Box.set_index('index')
print(Box)


boxLow=Box.at["boxLow",'geometry']
pdfLow = gpd.GeoDataFrame({'geometry':[boxLow]},crs="EPSG:4326")
boxNormal=Box.at["boxNormal",'geometry']
pdfNormal = gpd.GeoDataFrame({'geometry':[boxNormal]},crs="EPSG:4326")
boxHigh=Box.at["boxHigh",'geometry']
pdfHigh = gpd.GeoDataFrame({'geometry':[boxHigh]},crs="EPSG:4326")
boxBorder=Box.at["boxBorder",'geometry']
pdBorder = gpd.GeoDataFrame({'geometry':[boxBorder]},crs="EPSG:4326")

#SJOIN
#high
dfHigh = gpd.sjoin(gdf, pdfHigh, how = "inner", op="intersects")
dfHigh = dfHigh.drop(['index_right'],axis=1)
if dfHigh.empty == False:
    dfHigh.to_file(pathDFHigh+"DFHigh_"+suffix+".geojson",driver='GeoJSON')
print("Locazioni all'interno del Box High:",len(dfHigh))

#low
dfLow = gpd.sjoin(gdf, pdfLow, how = "inner", op="intersects")
dfLow = dfLow.drop(['index_right'],axis=1)
if dfLow.empty == False:
    dfLow.to_file(pathDFLow+"DFLow_"+suffix+".geojson",driver='GeoJSON')
print("Locazioni all'interno del Box Low:", len(dfLow))

#normal
dfNormal = gpd.sjoin(gdf, pdfNormal, how = "inner", op="intersects")
dfNormal = dfNormal.drop(['index_right'],axis=1)
if dfNormal.empty == False:
    dfNormal.to_file(pathDFNormal+"DFNormal_"+suffix+".geojson",driver='GeoJSON')
print("Locazioni all'interno del Box Normal:",len(dfNormal))

#Border
dfBorder = gpd.sjoin(gdf, pdBorder, how = "inner", op="intersects")
dfBorder = dfBorder.drop(['index_right'],axis=1)
if dfBorder.empty == False:
    dfBorder.to_file(pathDFBorder+"DFBorder_"+suffix+".geojson",driver='GeoJSON')
print("Locazioni all'interno del Box Boder:",len(dfBorder))



   Unnamed: 0   id_location        lat         lon  probability
0           0  1.889810e+09  39.725721  115.955237     0.021609
1           1  1.889810e+09  39.725248  115.955203     0.023833
2           2  1.938419e+09  39.748093  115.952030     0.000095
3           3  1.984411e+09  39.646400  115.841170     0.000000
4           4  1.984411e+09  39.720002  115.931293     0.000000
   Unnamed: 0   id_location  probability                    geometry
0           0  1.889810e+09     0.021609  POINT (115.95524 39.72572)
1           1  1.889810e+09     0.023833  POINT (115.95520 39.72525)
2           2  1.938419e+09     0.000095  POINT (115.95203 39.74809)
3           3  1.984411e+09     0.000000  POINT (115.84117 39.64640)
4           4  1.984411e+09     0.000000  POINT (115.93129 39.72000)
                 Area                                           geometry
index                                                                   
boxLow     94336596.0  POLYGON ((116.10000 40.30000, 116

# Opzionale
# ampliamento del dfLow con posizione e probabilità generate in maniera casuale e salvataggio su file

In [13]:
#aggiunta dfLow
boundsBox = boxLow.bounds
minlat=boundsBox[1]
maxlat=boundsBox[3]
minlon=boundsBox[0]
maxlon=boundsBox[2]

probability = np.random.uniform(0,0.6,250)
xpoints = np.random.uniform(0,0.1,250) + minlon
ypoints = np.random.uniform(0,0.1,250)+minlat
d = {'probability':probability,'geometry': gpd.points_from_xy(xpoints,ypoints)}
df = gpd.GeoDataFrame(d,crs="EPSG:4326")
dfLow=dfLow.append(df)
dfLow.to_file(pathDFLow+"DFLow.geojson",driver='GeoJSON')
dfLow


,Unnamed: 0,id_location,probability,geometry
8,8.0,300533008.0,0.799160,POINT (116.01036 40.34366)
9,9.0,822489029.0,0.309417,POINT (116.02020 40.35102)
0,NaN,NaN,0.004722,POINT (116.00451 40.39874)
1,NaN,NaN,0.349469,POINT (116.06413 40.33967)
2,NaN,NaN,0.440215,POINT (116.00470 40.38606)
...,...,...,...,...
245,NaN,NaN,0.453691,POINT (116.07460 40.31728)
246,NaN,NaN,0.028927,POINT (116.08189 40.38843)
247,NaN,NaN,0.169728,POINT (116.06555 40.37180)
248,NaN,NaN,0.131999,POINT (116.02061 40.30047)


# ampliamento del dfNormal con posizione e probabilità generate in maniera casuale e salvataggio su file

In [14]:
#aggiunta dfNormal
boundsBox = boxNormal.bounds
minlat=boundsBox[1]
maxlat=boundsBox[3]
minlon=boundsBox[0]
maxlon=boundsBox[2]

probability = np.random.uniform(0,1,200)
xpoints = np.random.uniform(0,0.1,200) + minlon
ypoints = np.random.uniform(0,0.1,200)+minlat
d = {'probability':probability,'geometry': gpd.points_from_xy(xpoints,ypoints)}
df = gpd.GeoDataFrame(d,crs="EPSG:4326")
dfNormal=dfNormal.append(df)
dfNormal.to_file(pathDFNormal+"DFNormal.geojson",driver='GeoJSON')
dfNormal

,Unnamed: 0,id_location,probability,geometry
128,128.0,7.114100e+09,0.054485,POINT (116.69887 40.27719)
129,129.0,2.372644e+08,0.174800,POINT (116.64019 40.31523)
0,NaN,NaN,0.662346,POINT (116.66475 40.27382)
1,NaN,NaN,0.519941,POINT (116.68656 40.27157)
2,NaN,NaN,0.327615,POINT (116.65711 40.27971)
...,...,...,...,...
195,NaN,NaN,0.804371,POINT (116.63822 40.28900)
196,NaN,NaN,0.208436,POINT (116.63694 40.31142)
197,NaN,NaN,0.869352,POINT (116.64786 40.34186)
198,NaN,NaN,0.961494,POINT (116.67325 40.32314)


# ampliamento del dfHigh con posizione e probabilità generate in maniera casuale e salvataggio su file

In [15]:
#aggiunta dfHigh
boundsBox = boxHigh.bounds
minlat=boundsBox[1]
maxlat=boundsBox[3]
minlon=boundsBox[0]
maxlon=boundsBox[2]

augemented_locations = 1000
probability = np.random.uniform(0,0.6,augemented_locations)
xpoints = np.random.uniform(0,0.1,augemented_locations) + minlon
ypoints = np.random.uniform(0,0.1,augemented_locations)+minlat
id_locations = np.random.uniform(1,100000,augemented_locations)

d = {'probability':probability,'geometry': gpd.points_from_xy(xpoints,ypoints),'id_location':id_locations}
df = gpd.GeoDataFrame(d,crs="EPSG:4326")
dfHigh=dfHigh.append(df)
dfHigh.to_file(pathDFHigh+"DFHigh.geojson",driver='GeoJSON')
dfHigh

,Unnamed: 0,id_location,probability,geometry
131,131.0,9061.138407,0.415942,POINT (116.33802 39.97296)
132,132.0,12842.470039,0.399887,POINT (116.33603 39.99617)
133,133.0,1302.119771,0.340016,POINT (116.32512 39.97085)
134,134.0,9196.924629,0.219219,POINT (116.30718 39.92141)
135,135.0,8646.317924,0.007257,POINT (116.36425 39.99985)
...,...,...,...,...
995,NaN,35287.438026,0.491708,POINT (116.30995 39.94224)
996,NaN,8246.901051,0.321583,POINT (116.39838 39.97669)
997,NaN,66004.744042,0.195652,POINT (116.31603 39.95456)
998,NaN,56209.814100,0.247069,POINT (116.39137 39.98884)
